In [1]:
import tabula


In [2]:
ficha_javi = "./fichas/20-0015-01-1080035-1-1.pdf"
ficha_abi = "./fichas/20-0025-00-1096523-1-1.pdf"

dfs_javi = tabula.read_pdf(ficha_javi, pages='all')
dfs_abi = tabula.read_pdf(ficha_abi, pages='all')

In [3]:
dfs_javi[4]

,#,Código\rUNSPS\rC,Objeto\rde\rGasto,Descripción del Bien o\rServicio,Unidad\rde\rMedida,Cantida\rd,Precio\rReferencial\rUnitario,Precio\rReferencial\rTotal,Nombre o Razón Social\rdel Proponente\rAdjudicado,Precio\rUnitario\rAdjudicado,Total\rAdjudicado,Cantida\rd\rRecepci\ronada,Precio\rUnitario\rReal,Monto Real\rejecutado,Estado,Cronograma,Unnamed: 0
0,1,4214000,34200,"ALCOHOL ETILICO\rDESINFECTANTE AL 70% vv,\rANT...",LITROS,"15,000.0",9.0,"135,000.00",INGENIO\rSUCROALCOHOLERO\rAGUAI S.A..,8.99,"134,850.00","15,000.00",8.99,"134,850.00",Contratado,Cronograma 0,NaN
1,Total Referencial:,"135,000.00",Total Adjudicado:,"134,850.00",Total Recepcionado:,"134,850.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dfs_abi[4]

,#,Código\rUNSPS\rC,Objeto\rde\rGasto,Descripción del Bien o\rServicio,Unidad\rde\rMedida,Cantida\rd,Precio\rReferencial\rUnitario,Precio\rReferencial\rTotal,Nombre o Razón Social\rdel Proponente\rAdjudicado,Precio\rUnitario\rAdjudicado,Total\rAdjudicado,Cantida\rd\rRecepci\ronada,Precio\rUnitario\rReal,Monto Real\rejecutado,Estado,Cronograma,Unnamed: 0
0,1,4214000,34500,BARBIJO KN 95,PQ/10\rUNID,300.00,55.0,"16,500.00",NaN,NaN,NaN,300.00,39.70,"11,910.00",Contratado,Cronograma 0,NaN
1,2,4214000,34500,BARBIJO DESECHABLE,PQ/50\rUNID,"1,000.00",25.0,"25,000.00",NaN,NaN,NaN,"1,000.00",24.47,"24,470.00",Contratado,Cronograma 0,NaN
2,3,4214000,34200,ALCOHOL AL 70%,FRASCO,500.00,12.5,"6,250.00",NaN,NaN,NaN,500.00,12.47,"6,235.00",Contratado,Cronograma 0,NaN
3,Total Referencial:,"47,750.00",Total Recepcionado:,"42,615.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
